In [1]:
import re
import json
import glob
import csv

In [194]:
files_feature = [xfile for xfile in glob.glob("data/Features/*.json")]
fileprefs = [re.findall('data/Features/(.*)_features_(.*)docs_(.*)labels_at(.*).json',ff)[0] for ff in files_feature]
mostrecent = [file for file in fileprefs if file[3] == max([date[3] for date in fileprefs])][0]
mostrecent

('EIFD-FlatData', '37', '1424', '2017-02-17T14:05:00')

In [195]:
with open('data/Features/' + mostrecent[0] + '_features_' + mostrecent[1] + 'docs_' 
          + mostrecent[2] + 'labels_at' + mostrecent[3] + '.json', 'r') as f:
    featlist = json.load(f)

In [196]:
for i,feat in enumerate(featlist):
    for key,val in feat.items():
        if(val==''):
            featlist[i][key]="None"

In [197]:
# Included classes
includedclasses = ["O","B.event.flood", "I.event.flood"]
for feat in featlist:
    if(feat["class"] not in includedclasses):
        feat["class"] = "O"
len(featlist)

31978

In [159]:
featlist = [feat for feat in featlist if feat["class"] not in ["O","B.dmg.ppl.miss","B.dmg.bus","I.dmg.bus","I.dmg.ppl.miss","B.loc.supp","I.loc.supp","B.loc.other","I.loc.other","B.resp.supply","I.resp.supply","B.dmg.econ","I.dmg.econ"]]
len(featlist)

1184

In [198]:
splt = int((80 * len(featlist)) / 100.0)
splt

25582

In [199]:
trainlist = featlist[:splt]
testlist = featlist[splt:]

In [200]:
notUsedFeatures = ["class", "foliaId"]#,"isTitleCase","wordClass","+1:wordClass","-1:wordClass","-2:ner","+2:ner","+2:pos","-2:pos","-1:pos","+1:pos","pos"]

In [201]:
timblefile= open('data/TimblData/flooddata.train', "w")

for i,feat in enumerate(trainlist):
    for key,val in feat.items():
        if(key not in notUsedFeatures):
            #print(key +'\t\t'+ val)
            timblefile.write(trainlist[i][key] + '\t')
    timblefile.write(trainlist[i]["class"] + '\n')


timblefile.close()

In [202]:
timblefile= open('data/TimblData/flooddata.test', "w")

for i,feat in enumerate(testlist):
    for key,val in feat.items():
        if(key!="class" and key!="foliaId"):
            #print(key +'\t\t'+ val)
            timblefile.write(testlist[i][key] + '\t')
    timblefile.write(testlist[i]["class"] + '\n')


timblefile.close()

In [203]:
i=1
for key,val in featlist[0].items():
    if(key not in notUsedFeatures):
        print(i,key)
        i += 1

1 -1:token|token|+1:token
2 -2:ner
3 ner|+1:ner
4 -2:token|-1:token|token
5 ner|+1:ner|+2:ner
6 token
7 -1:wordClass
8 token|+1:token|+2:token
9 -1:token|token
10 -1:ner
11 -2:token
12 -1:pos
13 -2:ner|-1:ner|ner
14 -1:token
15 -1:pos|pos|+1:pos
16 token|+1:token
17 wordClass
18 pos|+1:pos
19 +1:wordClass
20 pos
21 isTitleCase
22 -2:pos|-1:pos|pos
23 +1:pos
24 ner
25 pos|+1:pos|+2:pos
26 +2:token
27 -2:pos
28 -1:ner|ner
29 +1:token
30 -1:pos|pos
31 -1:ner|ner|+1:ner
32 +2:ner
33 +1:ner
34 +2:pos
